In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

2024-01-03 18:32:15.641789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Predictions (deep learning model)

In [2]:
df = pd.read_csv("embeddingsCTXencode_plus_Sbert.csv") # embeddings
df2 = pd.read_json("dataset.json") # targets

In [3]:
df2 = df2.iloc[1:] # remove first line because the embedding is title (TODO: fix csv)

In [4]:
print(df.shape, df2.shape)

(148121, 1536) (148121, 6)


In [5]:
X = np.array(df.values.tolist())
Y = pd.get_dummies(df2['category']).values  # One-hot encoding of categories

print(X.shape, Y.shape)

(148121, 1536) (148121, 15)


In [12]:
# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(Y.shape[1], activation='softmax')  # Prediction head
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X_test)
predicted_categories = np.argmax(predictions, axis=1)

# Display predictions
for pred, y in zip(predicted_categories[:100], np.argmax(Y_test, axis=1)):
    print(f"Target: {df2['category'].unique()[y]}, Prediction: {df2['category'].unique()[pred]}, Correctness: {pred == y}")


Epoch 1/10
3703/3703 [==============================] - 62s 16ms/step - loss: 0.9531 - accuracy: 0.7129
Epoch 2/10
3703/3703 [==============================] - 61s 16ms/step - loss: 0.7525 - accuracy: 0.7707
Epoch 3/10
3703/3703 [==============================] - 61s 16ms/step - loss: 0.6857 - accuracy: 0.7884
Epoch 4/10
3703/3703 [==============================] - 65s 18ms/step - loss: 0.6349 - accuracy: 0.8006
Epoch 5/10
3703/3703 [==============================] - 66s 18ms/step - loss: 0.5973 - accuracy: 0.8114
Epoch 6/10
3703/3703 [==============================] - 65s 18ms/step - loss: 0.5648 - accuracy: 0.8188
Epoch 7/10
3703/3703 [==============================] - 67s 18ms/step - loss: 0.5368 - accuracy: 0.8267
Epoch 8/10
3703/3703 [==============================] - 66s 18ms/step - loss: 0.5113 - accuracy: 0.8322
Epoch 9/10
3703/3703 [==============================] - 66s 18ms/step - loss: 0.4881 - accuracy: 0.8391
Epoch 10/10
926/926 [==============================] - 4s 4ms/st

In [13]:
N = len(predicted_categories)
c = 0
for pred, y in zip(predicted_categories, np.argmax(Y_test, axis=1)):
    if pred == y:
        c += 1
print(c, N)
print("ACC", c/N)

23772 29625
ACC 0.8024303797468354
